<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/99.Machine_Learning/19_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Naive Bayes



## Teorema de Bayes



<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Bayes%27_Theorem_MMB_01.jpg/1024px-Bayes%27_Theorem_MMB_01.jpg">
El teorema de Bayes (o Ley de Bayres o Regla de Bayes) se considera una de las reglas más poderosas en el cálculo de probabilidades y estadística.
Describa la probabilidad de un evente, basado en conocimiento previo de condiciones que podrían estar relacionadas con el evento.

 
<img src="https://miro.medium.com/max/1994/1*CnoTGGO7XeUpUMeXDrIfvA.png">

<img src="https://www.saedsayad.com/images/Bayes_rule.png">


- P(c|x) es la probabilidad a posteriori de la clase c dado los predictores x.
- P(c) es la probabilidad a priori de la clase.
- P(x) es la probabilidad a priori del predictor (o predictores).
- P(x|c) es la probabilidad del predictor, dada la clase (*likelihood*).



## Naive Bayes



Los algoritmos de Naive Bayes son algoritmos de aprendizaje supervisado basados en la aplicación del Teorema de Bayes con una suposición *naive* (inocente): **asume independencia** entre cada par de variables de predicción.



- Uso más común: como algoritmo de clasificación. Predice la clase de los datos.
- Ventaja de Naive Bayes: es relativamente rápido comparado con otros algoritmos de clasificación.
- Útil con datasets grandes.
- Aplicaciones: predicciones en tiempo real, predicciones multiclase, clasificación de textos / filtrado de spam / analisis de sentimientos, sistema de recomendaciones...



### Naive Bayes con Scikit Learn



Los diferentes algoritmos de Naive Bayes difieren en las suposiciones relativas a la distribución de las variables continuas predictoras.

En **Scikit** Learn tenemos:
- **GaussianNB**: implementa el algoritmo Gaussian Naive Bayes para clasificación. Asume que la distribución de los datos es gaussiana (normal).
- **MultinomialNB**: implementa el algoritmo de naive Bayes para datos dcon una distribución multinomial. Útil cuando las variables son discretas.
- **BernoulliNB**: impelmenta naive Bayes para datos con una distribución de Bernoulli multivariante. Útil cuando las variables son binarias.



Antes de utilizar un modelo, deberíamos verificar si la distribución de las variables sigue uno de los tres modelos recien mencionados. Porque si no lo hiciese, podríamos ver que las distribuciones no ajustan bien los datos.

Si las variables no siguen una distribución normal por ejemplo, podríamos usar transformaciones u otros métodos para convertirlas en una distribución normal.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
data = spark.read.csv(DATA_PATH+"titanic_train.csv", header=True, 
                      inferSchema=True).toPandas().set_index('PassengerId', inplace=True)

# Local
# data = pd.read_csv("data/titanic_train.csv",index_col="PassengerId")

In [ ]:
CAge=pd.cut(data["Age"], bins = [0,10,18,40,max(data["Age"])] ,labels=["Child","MYoung","Young","Older"])

#create dummy variables for the column
dummies = pd.get_dummies(CAge)



En la versión 0.19.0 de pandas es necesario un paso adicional

In [ ]:
dummies_tmp = pd.get_dummies(CAge)
dummies = pd.DataFrame(data = dummies_tmp.values,
                       index = dummies_tmp.index)
dummies.index.name="PassengerId"



añadimos las variables *dummy*

In [ ]:
data = data.join(dummies)

In [ ]:
data['Sex'] = data['Sex'].replace(to_replace=['male', 'female'], value=[0, 1])

In [ ]:
train, test = train_test_split(data, test_size=0.5)

survived = train[(train.Survived==1)]

In [ ]:
train.head()



## Prediciendo con la variable Fare

In [ ]:
fig7, axs7 = plt.subplots(1,2)

train['Fare'].plot.hist(ax=axs7[0], figsize=(12, 6), fontsize=16)
survived['Fare'].plot.hist(ax=axs7[1], fontsize=16)

axs7[0].set_title("People on the Titanic", fontsize=20)
axs7[1].set_title("People who survived", fontsize=20)

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [ ]:
gnb = GaussianNB()

used_features = ['Fare']

y_pred = gnb.fit(train[used_features], train["Survived"])

y_pred = gnb.predict(test[used_features])



Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

acc = accuracy_score(test["Survived"], y_pred)
acc

In [ ]:
print(confusion_matrix(test["Survived"], y_pred))



## Prediciendo con la variable Age

In [ ]:
fig3, axs3 = plt.subplots(1,2)

train['Age'].plot.hist(ax=axs3[0], figsize=(12, 6), fontsize=16)
survived['Age'].plot.hist(ax=axs3[1], fontsize=16)

axs3[0].set_title("People on the Titanic", fontsize=20)
axs3[1].set_title("People who survived", fontsize=20)

In [ ]:
used_features = ["Child","MYoung","Young","Older"]

In [ ]:
gnb = BernoulliNB()

y_pred = gnb.fit(train[used_features], train["Survived"])

y_pred = gnb.predict(test[used_features])

acc = accuracy_score(test["Survived"], y_pred)
acc

In [ ]:
print(confusion_matrix(test["Survived"], y_pred))



## Prediciendo con la variable Sex

In [ ]:
fig2, axs2 = plt.subplots(1,2)

train['Sex'].value_counts().sort_index().plot.bar(ax=axs2[0], figsize=(12, 6), fontsize=16)
survived['Sex'].value_counts().sort_index().plot.bar(ax=axs2[1], fontsize=16)

axs2[0].set_title("People on the Titanic", fontsize=20)
axs2[1].set_title("People who survived", fontsize=20)


In [ ]:
used_features = ["Sex"]

gnb = BernoulliNB()

y_pred = gnb.fit(train[used_features], train["Survived"])

y_pred = gnb.predict(test[used_features])

acc = accuracy_score(test["Survived"], y_pred)
acc


In [ ]:
print(confusion_matrix(test["Survived"], y_pred))



## Notas

- Remover las variables correlacionadas puede mejorar el modelo
- Laplace Correction: si una variable categórica (en los datos de test), tiene una categoría no observada en los datos de entrenamiento, el modelo le asignará una probabilidad de 0, y no podrá hacer una predicción. A esto se le llama *Zero Frequency*. Para remediarlo podemos usar una técnica llamada estimación de Laplace. 



# Naive Bayes casos prácticos

Veamos algunos ejemplos aplicando el teorema de Bayes



<center><img src="https://www.saedsayad.com/images/Bayes_rule.png"></center>



Supongamos unos datos como los de la imagen inferior, en la que tenemos las condiciones metereológicas (Weather) y la variable a predecir (Play) que representa el que un equipo salga a jugar o no.



<center><img src="https://www.saedsayad.com/images/naive_bayes_data.png"></center>



El primer caso consiste en convertir los datos anteriores a una tabla de frecuencia y luego creamos las tablas de likelihood, donde tenemos las probabilidades de cada weather y de Play (Yes/No)



Mediante estas tablas y  Naive Bayes podemos calcular la probabilidad por ejemplo de si se **jugará (Play=YES) si el tiempo es Sunny**. En nuestro caso, quedaría así la ecuación planteda inicialmente:


                             P(Yes | Sunny) = P( Sunny | Yes) * P(Yes) / P (Sunny)
                                                        
                                                        
Donde tenemos, de las tablas anteriores:

                                         P (Sunny |Yes) = 3/9 = 0.33
                                         P(Sunny) = 5/14 = 0.36
                                         P(Yes)= 9/14 = 0.64

Lo que nos queda sustituyendo:
                                                                 
                                 P (Yes | Sunny) = 0.33 * 0.64 / 0.36 = 0.60
                                                                 




<center><img src="https://www.saedsayad.com/images/Bayes_3.png"></center>



# Ejercicio 

El teorema de Naive Bayes se encuentra implementado en scikit-learn:

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB



Con lo que se puede podemos construir un clasificador de correo ham/spam que obtiene buenos resultados.

Dado el dataset 'data/spam.csv', construya un clasificador de correo con el que predecir si el correo es ham o spam.


Nota: La estructura inicial del dataset consiste en el contenido del correo y una etiqueta que lo identifica como ham/spam. Es necesario transformar el contenido del correo en variables numéricas que representen la frecuencia de aparición de cada término en el dataset.

Esta transformación se puede realizar mediante el siguiente código:



Dividimos el conjunto de datos iniciales en train y test, con 10% de test

In [ ]:
# Respuesta aqui



convertimos la estructura de texto en términos de frecuencia  de cada término

In [ ]:
# Respuesta aqui



seleccionamos aquellos términos más frecuentes

In [ ]:
# Respuesta aqui



- a. Construya un clasificador con sklearn usando las distintas opciones (GaussianNB, MultinomialNB, BernoulliNB) y evalúe el resultado mediante la accuracy y la matriz de confusión:

from sklearn.metrics import accuracy_score, confusion_matrix


In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui